# Import Packages

In [1]:
# we will import everything here
import numpy as np
import pygame
import random
from pygame.locals import *
from pandas import *

from src.read_file import *
from src.parser import *
from src.simulation import *
from src.dijkstra import dijkstra

%reload_ext autoreload
%autoreload 2

pygame 2.1.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Task 1/5: Setting up the modeling environment

In [2]:
# read the initial state of the cellular automata from a file
X = read_file("input.txt")
print("Input state matrix: ")
print(X)

# parse the input and get the required parameters
rows, columns, pedestrians, obstacles, targets = parser(X)

print("number of rows and columns")
print(rows, columns)

print("coordinate list of the pedestrians")
print(pedestrians)

print("coordinate list of the obstacles")
print(obstacles)

print("coordinate list of the targets")
print(targets)

Input state matrix: 
[[0 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 0 2 0 0]
 [1 0 2 2 0 0]
 [1 0 2 3 0 0]
 [0 0 2 2 0 0]]
number of rows and columns
6 6
coordinate list of the pedestrians
[[3 0]
 [4 0]]
coordinate list of the obstacles
[[1 1]
 [2 3]
 [3 2]
 [3 3]
 [4 2]
 [5 2]
 [5 3]]
coordinate list of the targets
[[4 3]]


# Task 2/5: First step of a single pedestrian

In [12]:
#initiliaze colors for further use.
# red for pedestrian ,orange for obstacle, purple for obstacle
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
orange=(250,89,8)
purple=(210,8,250)

#set width and height for grid rectangles and the distance between rectangles
WIDTH = 25
HEIGHT = 25
MARGIN = 5

#initiliaze pygame , font for P O T , and render them to place in cells
pygame.init()
font = pygame.font.SysFont('Calibri',20, False, False)
pds = font.render("P",True,black)
obs = font.render("O",True,black)
tgt = font.render("T",True,black)

#window size is set according num of columns and rows , +1 because end and start lines should both be visible
window_size = [columns*WIDTH+(columns+1)*MARGIN,rows*HEIGHT + (rows+1)*MARGIN]
scr = pygame.display.set_mode(window_size)
pygame.display.set_caption("Simulator")
done = False
clock = pygame.time.Clock()

#main loop for simulation , if quit leave loop
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    #  elif event.type == pygame.MOUSEBUTTONDOWN:
    #  pos = pygame.mouse.get_pos()
    #  column = pos[0] // (WIDTH + MARGIN)
    #  row = pos[1] // (HEIGHT + MARGIN)
    #  grid[row][column] = 1
    #  print("Click ", pos, "Grid coordinates: ", row, column)

    #color background to black, they will then become the lines between rectangles
    scr.fill(black)

    grid = []
    # 0 = empty cell
    for row in range(rows):
        grid.append([])
        for column in range(columns):
            grid[row].append(0)

    # 1 = pedestrian in cell, 2 = obstacle in cell, 3 = target in cell
    for pedestrian in pedestrians:
        grid[pedestrian[0]][pedestrian[1]] = 1
    for obstacle in obstacles:
        grid[obstacle[0]][obstacle[1]] = 2
    for target in targets:
        grid[target[0]][target[1]] = 3

    # for each row and column draw  white rectangles
    for row in range(rows):
        for column in range(columns):
            color = white

            if grid[row][column] == 1:
                color = red
            if grid[row][column] == 2:
                color = purple
            if grid[row][column] == 3:
                color = orange

            pygame.draw.rect(scr,
                             color,
                             [(MARGIN + WIDTH) * column + MARGIN,
                              (MARGIN + HEIGHT) * row + MARGIN,
                              WIDTH,
                              HEIGHT])

            #put P,O,T
            if color ==red:
                 scr.blit(pds,[(MARGIN + WIDTH) * column + MARGIN,(MARGIN + HEIGHT) * row + MARGIN])
            if color ==purple:
                 scr.blit(obs,[(MARGIN + WIDTH) * column + MARGIN,(MARGIN + HEIGHT) * row + MARGIN])
            if color ==orange:
                 scr.blit(tgt,[(MARGIN + WIDTH) * column + MARGIN,(MARGIN + HEIGHT) * row + MARGIN])

    # update positions of pedestrians
    sim = Simulator(X, (targets[0][0], targets[0][1]))
    sim.update(X, pedestrians)
    rows, columns, pedestrians, obstacles, targets = parser(X)

    pygame.time.delay(2000)
    clock.tick(60)
    #bring the drawings to screen
    pygame.display.flip()
pygame.quit()

[ 9.         11.36787944  9.         10.36787944]
[10.36787944 12.         10.         11.36787944]
[10.         10.          8.          9.36787944]
[ 9.         11.         11.                 inf 10.36787944]
[       inf 9.36787944 9.         7.         8.36787944]
[ 8.36787944 10.         10.                 inf  9.36787944]
[       inf 9.         9.         7.36787944 8.36787944]
[6.                inf 8.36787944        inf 7.36787944]
[7.         7.                inf 5.         6.36787944]
[6.                inf 6.         4.         5.36787944]
[5.         3.         5.         5.         4.36787944]


KeyboardInterrupt: 

# Task 3/5: Interaction of pedestrians

# Task 4/5: Obstacle avoidance

In [3]:
# compute the distance matrix using Dijkstra's algorithm
D = dijkstra(X, targets[0])

# Making the dataframe for nicer print
#df_D = 
print("This is the distance matrix\n", DataFrame(D))

This is the distance matrix
     0     1    2    3  4  5
0   9   8.0  7.0  6.0  5  6
1  10   inf  6.0  5.0  4  5
2   9   8.0  7.0  inf  3  4
3  10   9.0  inf  inf  2  3
4  11  10.0  inf  0.0  1  2
5  12  11.0  inf  inf  2  3
